In [1]:
import copy
import os

import pandas as pd
import polib
import requests

# Generate file with the translations of all the creatures' forms in appropriate context

In [107]:
google_api_key = 

In [34]:
cells_range = 'A1:I1000'
creature_cells_range = f"Creatures!{cells_range}"

url_template = f"https://sheets.googleapis.com/v4/spreadsheets/1CLBuInJyZrWiFlbg5p0bT2QhdoZW1IemyB_lABODwMY/values/{{}}?key={google_api_key}"

In [26]:
creature_names_data = requests.get(url_template.format(creature_cells_range)).json()

In [28]:
creature_names_df = pd.DataFrame.from_records(
    creature_names_data['values'],
    columns=['#', 'msgid'],
).set_index("msgid")[1:]
creature_names_df

,#
msgid,
program bug,0
bumblebee,1
wood golem,2
ant larva,3
laboratory rat,4
...,...
friendly sensed monster,650
trivial sensed monster,651
easy sensed monster,652


In [35]:
creature_contexts_cells_range = f"'Creatures contexts'!{cells_range}"
creature_contexts_data = requests.get(url_template.format(creature_contexts_cells_range)).json()
forms = "the	a	the 's".split('\t')

In [37]:
creature_contexts_df = pd.DataFrame.from_records(
    creature_contexts_data['values'],
    columns=forms,
)[1:]
creature_contexts_df

,the,a,the 's
1,%s bites you for 10 damage with +3 dagger!!!,,%s sword falls to the floor!
2,The jackal bites %s for 10 damage with +3 dagg...,None,None
3,You slice %s like an onion for 10 damage!!,None,None
4,%s is blown up!,None,None
5,You kill %s!,None,None
6,%s hits a closed door.,None,None
7,%s loses its grip on you.,None,None
8,Octopode loses its grip on %s.,None,None
9,"%s constrict the jackal for 15, but do no damage.",None,None
10,"The grasping roots constrict %s for 15, but do...",None,None


In [102]:
vowels = set('euioa')


def _creature_mapper(form, creature):
    first_letter = creature[0]
    is_name = first_letter.isupper()

    if form == 'the':
        if is_name:
            result = creature
        else:
            result = f'the {creature}'
    elif form == 'a':
        if is_name:
            result = creature
        elif first_letter in vowels:
            result = f'an {creature}'
        else:
            result = f'a {creature}'
    elif form == "the 's":
        creature, result = _creature_mapper('the', creature)

        if creature[-1] == 's':
            result = f"{result}'"
        else:
            result = f"{result}'s"
    else:
        raise Exception(f'Unknown form: {form}')
    
    return creature, result
                                    
                                    
def _forms_gen(form):
    for creature in creature_names_df.index:
        yield _creature_mapper(form, creature)


def forms_gen():
    for form in forms:
        contexts = creature_contexts_df[form][lambda x: x != ''].dropna()
        words_forms = pd.DataFrame.from_records(_forms_gen(form), columns='creature form'.split()).set_index('creature')['form']

        yield form, contexts, words_forms
next(forms_gen())

('the', 1          %s bites you for 10 damage with +3 dagger!!!
 2     The jackal bites %s for 10 damage with +3 dagg...
 3            You slice %s like an onion for 10 damage!!
 4                                       %s is blown up!
 5                                          You kill %s!
 6                                %s hits a closed door.
 7                             %s loses its grip on you.
 8                        Octopode loses its grip on %s.
 9     %s constrict the jackal for 15, but do no damage.
 10    The grasping roots constrict %s for 15, but do...
 11          %s is constricted for 15, but do no damage.
 12                                 You collide with %s!
 13                                %s collides with you!
 14        Collided with %s caused by salamander mystic.
 15               Collided with the jackal caused by %s.
 16                              %s slams into the wall!
 17                           %s violently stops moving!
 18       %s completely 

In [103]:
import polib

In [104]:
translations_file = polib.POFile()

In [105]:
for form, contexts, words_forms in forms_gen():
    for context in contexts:
        for msgid in words_forms.index:
            translations_file.append(polib.POEntry(msgid=msgid, msgstr=words_forms[msgid], msgctxt=context))
translations_file.save('./monsters_in_contexts.po')